In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import csv
import re
import json
from time import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
url = 'https://www.1mg.com/sitemap.xml'

Html_Page = requests.get(url)
page_text = Html_Page.text
Html_parser =  bs(page_text,'html.parser')

Productdumps = Html_parser.find_all('loc')

In [ ]:

for dump in Productdumps:

    try:
        dump_text = dump.text
        ProductLink_html =  requests.get(dump_text).text
        ProductLink_parser = bs(ProductLink_html,'html.parser')
        ProductLinks = ProductLink_parser.find_all('loc')

        for Productlink in range(1,len(ProductLinks)):
            try:
                product = ProductLinks[Productlink]
                product = product.text

                response = requests.get(product)
                html_content = response.content
                Html_product_parser = bs(html_content,'html.parser')
                Json_details = Html_product_parser.select('script',{'nonce':'7a8a5f1c1b7543545fcd5f9eabea9406'})
                product_Detail = Json_details[30].text
                Data = re.search(r'window.__INITIAL_STATE__ = ({.*});', product_Detail).group(1)

                ProductData = json.loads(Data)

                AllDetails = {}
                try:
                    StaticData = ProductData['drugPageReducerV2']['staticData']
                    sku_summary = StaticData['sku']['summary']
                    DynamicData = ProductData['drugPageReducerV2']['dynamicData']
                    productSubstitutesData = DynamicData['productSubstitutes']['attributesData']
                except:     
                     continue
                
                try:
                    AllDetails['ProdutName'] = bs(StaticData['pageTitle']['header']).text
                except:
                    continue
                try:
                    AllDetails['Description'] = bs(StaticData['pageTitle']['description']).text
                except:
                    continue
                try:
                    AllDetails['Product_use'] = bs(StaticData['productUses']['content']).text.strip()
                except:
                    continue
                try:
                    AllDetails['productBenefits'] = bs(StaticData['productBenefits']['content']).text
                except:
                    continue
                try:
                    AllDetails['sideEffect'] = bs(StaticData['sideEffect']['content']).text
                except:
                    continue
                try:
                    AllDetails['Combination'] = bs(sku_summary['salt_composition']['display_text']).text
                except:
                    continue
                try:
                    AllDetails['manufacturer'] =bs(StaticData['sku']['manufacturer']['name']).text

                except:
                    continue

                try:
                    AllDetails['MRP'] = bs(DynamicData['priceBox']['priceList'][0]['mrp']['price']).text
                except:
                    continue
                try:
                    AllDetails['Discount'] = DynamicData['priceBox']['priceList'][1]['displayText']
                except:
                    continue
                try:
                    AllDetails['SR'] = DynamicData['priceBox']['priceList'][1]['price']
                except:
                    continue
                try:
                    productSubstitutesData = DynamicData['productSubstitutes']['attributesData']
                    AllDetails['subs'] = ",".join(bs(productSubstitutesData[i]['header']).text for i in range(1,4))
                except:
                    continue

                try:
                    AllDetails['PageLink'] = product
                except:
                    continue

                with open('1MG.csv','a',newline="") as file:
                        writer = csv.DictWriter(file, fieldnames=AllDetails.keys())
                        writer.writerow(AllDetails)


            except Exception as e:
                continue


    except Exception as e:
        continue
        
        
